In [58]:
import os
import google.generativeai as genai
import spacy
import spacy_llm
from spacy_llm.registry import registry
from spacy.tokens import Doc
from typing import Iterable
from transformers import pipeline
import torch

In [59]:
!pip install spacy_llm

In [60]:
!pip install google-generativeai

In [61]:
from google.colab import userdata

In [62]:
gemini_key = userdata.get('GEMINI_API')


In [63]:
class GeminiModel:
    def __init__(self, api_key: str):
        print(f"Initializing Gemini...")
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel("gemini-2.5-flash-lite")
        print("Gemini ready")

    def __call__(self, prompts: Iterable[str]):
        responses = []
        for prompt in prompts:
            if isinstance(prompt, list):
                prompt = prompt[0]
            response = self.model.generate_content(prompt)
            responses.append(response.text.strip())
        return responses

In [64]:
@registry.llm_models("custom.Gemini.v1")
def gemini_model(api_key: str):
    return GeminiModel(api_key)

In [65]:
@registry.llm_tasks("custom.TripletExtraction.v1")
def make_triplet_task():
    return TripletExtractionTask()

class TripletExtractionTask:
    def __init__(self):
        if not Doc.has_extension("triplets"):
            Doc.set_extension("triplets", default=[])

    def generate_prompts(self, docs: Iterable[Doc]) -> Iterable[str]:
        prompts = []
        for doc in docs:
            prompt = f"""Extract role-practice-counterrole triplets from this text.

IMPORTANT RULES:
- Role: Name of organization or actor (e.g., "EIT Digital", "Microsoft")
- Practice: SINGLE ACTION VERB ONLY (e.g., "helps", "funds", "collaborates", "supports")
- Counterrole: Name of partner or recipient (e.g., "MatchX", "startups")

Format each triplet EXACTLY like this:
Role: [organization name]
Practice: [single verb]
Counterrole: [partner name]

Text: {doc.text}

Triplets:"""
            prompts.append(prompt)
        return prompts

    def parse_responses(self, docs: Iterable[Doc], responses: Iterable[str]):
        docs_list = list(docs)
        responses_list = list(responses)

        for doc, response in zip(docs_list, responses_list):
            triplets = []
            current = {}
            for line in response.strip().split('\n'):
                if line.startswith('Role:'):
                    current = {'role': line.replace('Role:', '').strip()}
                elif line.startswith('Practice:'):
                    current['practice'] = line.replace('Practice:', '').strip()
                elif line.startswith('Counterrole:'):
                    current['counterrole'] = line.replace('Counterrole:', '').strip()
                    triplets.append(current)
                    current = {}
            doc._.triplets = triplets

        return iter(docs_list)

In [66]:
config = {
    "task": {"@llm_tasks": "custom.TripletExtraction.v1"},
    "model": {
        "@llm_models": "custom.Gemini.v1",
        "api_key": gemini_key
    },
    "validate_types": False
}

In [67]:
text = """I believe in the European dream
In his family, he has been the only one with an interest in technology:
'I had to teach myself how to
Alexandros Toptsoglou wants to return one day and apply smart technology to improve the lives of Greek people. First, he must finish his studies at the
Toptsoglou's self-motivation has culminated with the
EIT Digital Master School
. Toptsoglou is studying in the Cyber Security programme. During his first year, he studied at Twente University of Technology and now he is doing his second and final year at Saarland University in Germany. Before this, he studied digital systems at the University of Piraeus where he specialised in telecommunications and networks, writing a bachelor thesis on cloud forensics, and completing an Erasmus in Leuven.
Why did you choose to study at the EIT Digital Master School?
I have always been looking for opportunities to travel abroad. While doing my Erasmus, I realised that I'm interested in cyber security and that I have no problem adapting to new countries. The EIT Digital Master School combines both cyber security and travelling. Choosing the EIT Digital Master School also seemed good for my life after graduation. I think the network it provides will help me find a job more easily. In addition, I get a double degree from two universities and a certificate from the European Institute of Innovation and Technology (EIT). EIT is a recognised European Institute and the universities involved are among the highest rated worldwide in their fields. Also, the minor in Innovation and Entrepreneurship is quite special. Now that I am almost done, I am truly sure that I made the right choice!
What do you like about the EIT Digital Master School?
The multicultural diversity. You live and study with international students and see how differently everyone thinks. It is good to have discussions; it sharpens your critical and technical thinking and you become much more open-minded. The tough part is that sometimes you cannot express yourself the way you want, because of the language. I want to work abroad, so I think this is a good preparation for my career; the fact that I spent two years in two different universities and learned how to adapt to new environments. I saw how two highly-rated universities approach education and I believe that even if I faced difficulties in the beginning, it is worth it, and it will pay off. Finally, [tweetable alt="the structure of the EIT Digital Master School programme helped me to become more adaptable to change, not only in academic situations, whic"]the structure of the EIT Digital Master School programme helped me to become more adaptable to change, not only in academic situations, which is a key advantage for my future.[/tweetable]
What are the key differences between the different university systems you have experienced?
The system in the Netherlands is pressurising. I tried to adjust to this term-based system, which involves exams every eight weeks. That was intense and new to me, since in Greece we follow semesters. In Twente, they did not have a lot of lectures, so I had to study a lot of research papers myself to be prepared for the exams. Nevertheless, by organising my time efficiently, I managed to get used to it. In Saarland, the system is totally different to the Netherlands; they follow the semester system, similar to Greece. Here the lectures are given inside internationally recognised research centres such as Max Planck Institute and CISPA and this influences the lectures since modern research is served even before its publication.
In between the two universities, you did the Summer School Digital Finance Economics of Blockchain Security in Budapest. How did you find it?
My thoughts are only positive. The most exciting point is that I worked in a team where the members met for the first time. That was difficult at first since we all came from different backgrounds, both cultural and educational. Nevertheless, with determination, we managed to overcome all our differences and develop a business plan for our product based on blockchain, which we presented to a jury.
In addition to this, I enjoyed meeting other EIT Digital students from different disciplines. I realised that most of us want to be part of a start-up and that the Summer School provided us with all the necessary information to maximise our chances of building a successful product and a successful start-up.
Finally,[tweetable alt="I am also glad because once again EIT Digital gave me the chance to travel to a new country and meet new people and, most importantly, prepa"] I am also glad because once again EIT Digital gave me the chance to travel to a new country and meet new people and, most importantly, prepared me for the future work environment.[/tweetable] In the future, I will surely be involved in team projects with strict deadlines and people I don't know.
Why did you not choose to do a masters in Greece?
The most important reasons were that I wanted to see how they teach in other European countries and I wanted to broaden my horizons. Another reason is that studying for a masters in Greece is very expensive. In addition to this, due to the bad economic situation, universities, which are all public, are under-funded and consequently, they think twice when it comes to investing money in research and technology. When you are interested in more state of the art technologies, going abroad is a must. The last reason is that I would like to look for a job abroad.
What are your plans after your graduation?
Before I started at the EIT Digital Master School, I had a clear dream that I wanted to work in a big company or organisation in security or privacy and be the technical guy. After the kick-off in Rennes, and especially after the Summer School in Budapest, I started thinking about entrepreneurial things. I see how entrepreneurship works now and I can see myself in the future as an entrepreneur. Before this school, I was afraid to even think about starting something myself. Now I am for it. The ideal situation will be a job which combines my technical major with a taste of entrepreneurship.
Will you stay abroad or go back to Greece?
In Greece, we have a science brain drain. Greece produces brains but loses them. There is, for example, a large Greek community in Twente with people having serious technical jobs. They agree that Greece does not provide a stable work environment and is not friendly for young scientists. They want to come back but only after a few years. I also want to go back in maybe five or six years. I want to help and save my country at least from this brain drain but I cannot do that right now. I can easily find work, but the salary is not good. I also believe that being an entrepreneur in Greece is underestimated. First and foremost, the economy and the tax system cannot provide a safe environment for building a start-up and also the country makes it very dangerous with their economic tax policies. There is also a lack of vision and trust. Hence, starting a new business in Greece is impractical. For these reasons, I want to spend my first years abroad and after some years I will return with my savings to do my best for my country. In the future, I would like to apply smart technology and secure automated systems to reduce corruption, so we can save money for the government and make lives for people easier.
How will your study help you to help Greece?
Even if Greece is part of the European Union, it often lacks technology and innovation. Indeed, technology arrives in Greece years after it is applied in the rest of Europe. In a discipline such as cyber security, even a small delay in applying modern technology is crucial. Thus, this should be changed! I believe in the European dream, I believe that all countries should be together and communicate better to enable smooth technology transfer. My studies helped me to understand two things; first that people can communicate and work really successfully together even if they are from different countries; and second, I feel more open-minded now, since I have seen how other countries deal with the problems that Greece faces. When I return to my home country, I would like to try to teach, introduce and spread state of the art technology across industry. Finally, I would like to conduct research on a large scale and make Greece a pioneer in the field of cyber security.
to get the best of the EIT Community's news in your inbox once each month.
Do you use social media? Follow us and get updates in your news feed.
--- 2018/news_183.txt ---
Providing a broad portfolio of essential healthcare products,
is a global healthcare company.
Its portfolio includes acute and chronic dialysis therapies; sterile IV solutions; infusion systems and devices; parenteral nutrition therapies; inhaled anaesthetics; generic injectable pharmaceuticals; and surgical hemostat and sealant products. Baxter's global footprint and the critical nature of its products and services play a key role in expanding access to healthcare in emerging and developed countries. Baxter’s employees worldwide are building upon the company’s rich heritage of medical breakthroughs to advance the next generation of healthcare innovations that enable patient care.
Baxter operates a global manufacturing network and its products are sold in more than 100 countries around the world, and they are used in a variety of care settings, including hospitals, kidney dialysis centres, nursing homes, rehabilitation centres, doctors’ offices and by patients at home under physician supervision. Baxter maintains a global R&D footprint, including locations in the United States, Asia and in Europe, where we operate facilities in countries such as Belgium, Germany, Italy among others.
Baxter is thrilled to become a member of the
. They are looking forward to continuing to build open innovation partnerships which drive forward their mission of saving and sustaining lives, adding to recently announced collaborations like Mayo Clinic, Tel Aviv University/Tel Aviv Sourasky Medical Center, and MATTER Healthcare Incubator. Baxter brings to its partnerships a mix of broad experience across a range of products, therapies, and technologies, brand/product strength, global reach, and commercial, market, regulatory, and clinical expertise.
Baxter is particularly interested in opportunities/partnerships in the critical care/ICU, operating room, and renal care (home/clinic/hospital) settings. They are open to diverse partnerships and ones that cross industries lines.
to get the best of the EIT Community's news in your inbox once each month.
Do you use social media? Follow us and get updates in your news feed."""


In [68]:
nlp = spacy.blank("en")
nlp.add_pipe("llm", config=config)

Initializing Gemini...
Gemini ready


In [69]:
doc = nlp(text)

print("Extracted Triplets:")
print("=" * 60)
for i, t in enumerate(doc._.triplets, 1):
    print(f"\n{i}.")
    print(f"Role: {t['role']}")
    print(f"Practice: {t['practice']}")
    print(f"Counterrole: {t['counterrole']}")
print("\n" + "=" * 60)
print(f"Total triplets extracted: {len(doc._.triplets)}")

Extracted Triplets:

1.
Role: EIT Digital Master School
Practice: helps
Counterrole: Alexandros Toptsoglou

2.
Role: EIT Digital Master School
Practice: gave
Counterrole: Alexandros Toptsoglou

3.
Role: EIT Digital
Practice: gave
Counterrole: Alexandros Toptsoglou

4.
Role: Baxter
Practice: brings
Counterrole: Baxter's partnerships

Total triplets extracted: 4
